In [ ]:
from dataTable import *
from dash import *
import os

# 加载数据
A = MyDF.from_excel('TestData/A.xlsx', 'Sheet1', skipRows=[1])
B = MyDF.from_excel('TestData/B.xlsx', 'Sheet1', skipRows=[1])
diff = A.Compare(B)

# 转换 Pandas DataFrame 为 Dash DataTable
def PDToDash(df, max_width='1600px', highlight_diff=False):
    style_table = {
        'overflowX': 'auto',
        'overflowY': 'auto',
        'maxHeight': '500px',
        'maxWidth': max_width,
        'margin': '10px',
    }
    # 排除 _diff_cols 列
    columns = [{"name": i, "id": i} for i in df.columns if i != '_diff_cols']
    conditional_styles = []
    if highlight_diff and '_diff_cols' in df.columns and not df.empty:
        for idx, row in df.iterrows():
            diff_cols = row['_diff_cols']
            if isinstance(diff_cols, list) and diff_cols:
                for col in diff_cols:
                    if col in df.columns:
                        conditional_styles.append({
                            'if': {
                                'filter_query': f'{{{A.idField}}} = "{row[A.idField]}"',
                                'column_id': col
                            },
                            'backgroundColor': 'yellow',
                            'fontWeight': 'bold'
                        })
    return dash_table.DataTable(
        data=df.to_dict('records'),
        columns=columns,
        style_table=style_table,
        style_data_conditional=conditional_styles,
    )

# Dash 应用
app = Dash()

# 布局
textColor = 'grey'
app.layout = html.Div([
    html.H3("两表差异", style={'color': textColor}),
    html.Div([
        html.Div([
            html.Button('复制A表差异', id='diffA'),html.Button('打开A表', id='openA'),
            html.Div(A.path, style={'color': textColor}),
            PDToDash(diff['diffA'], max_width='900px', highlight_diff=True),
        ], style={'width': '50%', 'display': 'inline-block', 'verticalAlign': 'top'}),
        html.Div([
            html.Button('复制B表差异', id='diffB'),html.Button('打开B表', id='openB'),
            html.Div(B.path, style={'color': textColor}),
            PDToDash(diff['diffB'], max_width='900px', highlight_diff=True),
        ], style={'width': '50%', 'display': 'inline-block', 'verticalAlign': 'top'}),
    ], style={'display': 'flex', 'gap': '20px'}),
    html.H3("B表多出项", style={'color': textColor}),
    html.Button('复制B表多出项', id='add'),
    html.Div(B.path, style={'color': textColor}),
    PDToDash(diff['add']),
    html.H3("A表多出项", style={'color': textColor}),
    html.Button('复制A表多出除项', id='remove'),
    html.Div(A.path, style={'color': textColor}),
    PDToDash(diff['remove']),
]) 

# 回调函数
@callback(
    Input('diffA', 'n_clicks'),
    Input('openA', 'n_clicks'),
    Input('diffB', 'n_clicks'),
    Input('openB', 'n_clicks'),
    Input('add', 'n_clicks'),
    Input('remove', 'n_clicks'),
    prevent_initial_call=True
)
def OnInteract(a, openA, b, openB, add, remove):
    id = ctx.triggered_id
    if id == 'openA':
        os.system('start '+(A.path))
    elif id == 'openB':
        os.system('start '+(B.path))
    else:
        if not diff[id].empty:
            diff[id].drop(columns=['_diff_cols'], errors='ignore').to_clipboard(excel=True, sep=None, index=False, header=None)
        else:
            print(f"No data to copy for {id}")

if __name__ == '__main__':
    app.run(debug=False)
print("http://127.0.0.1:8050/")

TypeError: __init__() got multiple values for argument 'skipRows'